In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [2]:
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(f"Number of characters is: {len(text)}")

Number of characters is: 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
for i in range(30, 70):
    print(i,text[i])

30 e
31 d
32  
33 a
34 n
35 y
36  
37 f
38 u
39 r
40 t
41 h
42 e
43 r
44 ,
45  
46 h
47 e
48 a
49 r
50  
51 m
52 e
53  
54 s
55 p
56 e
57 a
58 k
59 .
60 

61 

62 A
63 l
64 l
65 :
66 

67 S
68 p
69 e


In [6]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
chars

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [8]:
s_to_i = {s:i for i,s in enumerate(chars)}
i_to_s = {i:s for i,s in enumerate(chars)}
encode = lambda s: [s_to_i[c] for c in s]
decode = lambda l: ''.join([i_to_s[i] for i in l])

print(encode("hello there"))
print(decode(encode("hii there")))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is {target}.")

When input is tensor([18]) the target is 47.
When input is tensor([18, 47]) the target is 56.
When input is tensor([18, 47, 56]) the target is 57.
When input is tensor([18, 47, 56, 57]) the target is 58.
When input is tensor([18, 47, 56, 57, 58]) the target is 1.
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15.
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47.
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58.


In [12]:
# torch.manual_seed(1337)
# batch_size = 4
# block_size = 8

# def get_batch(split):
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size, (batch_size,)) #[num, num, ... 8 times]
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     return x,y

# xb, yb = get_batch('train')
# print(f'Inputs: {xb} \nInput Shape: {xb.shape}')
# print(f"Targets: {yb} \nTarget Shape: {yb.shape}")
# print('-------')

# for b in range(batch_size): #batch dimension
#     for t in range(block_size): #time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"When input is {context.tolist()} the target is {target}")

In [13]:
#hyperparameters
batch_size = 32       #no. of independent seq we process in parallel
block_size = 8        #max context length for pred
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32

In [14]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #[num, num, ... 8 times]
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y =  x.to(device), y.to(device)
    return x,y

@torch.no_grad() # don't call .backward when doing estimate_loss()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out    

In [15]:
class Head(nn.Module):
    #one head of self-attention

    def __init__(self, head_size):
        super().__init__() #nn.Module's __init__
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) #lower triangular matrix of ones

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) #(B,T,head_size)
        q = self.query(x) #(B,T,head_size)
        wei = q @ k.transpose(-2,-1) * (k.shape[-1] **-0.5) #(B,T,head_size) @ (B,head_size,T) --> (B,T,T) *C**-0.5 is done to ensure that before softmax the variance is 1
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B,T,T) -inf so that softmax outputs 0 for them
        wei = F.softmax(wei, dim=-1) #(B,T,T)
        v = self.value(x) #(B,T,head_size)
        out = wei @ v #(B,T,head_size)
        return out

In [16]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__() #nn.Module's __init__
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)


    def forward(self, idx, targets=None):
        
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B,T,C) (number of batches or sequences in parallel, number of tokens being processed, embedding dimension)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = tok_emb + pos_emb #(B,T,C)
        x = self.sa_head(x) #apply one head of self-attention (B,T,C)
        logits = self.lm_head(x) #(B,T,C) here C is vocab size. C was 32 earlier now it is 65 as we want logits here

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond) #equivalent to self.forward(idx, targets=None) so (B,T,C)
            logits = logits[:, -1, :] #select last timestamp so it is (B,C), Given the seq so far, how likely is each possible next token
            probs = F.softmax(logits, dim=-1) #(B,C)
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
        
        return idx


xb, yb = get_batch('train')
model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([256, 65])
tensor(4.1689, device='cuda:0', grad_fn=<NllLossBackward0>)


In [17]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
     #every now and then evaluate the loss on train and val sets
     if iter % eval_interval == 0:
          losses = estimate_loss()
          print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
     xb, yb = get_batch('train')
     logits, loss = model(xb, yb)
     optimizer.zero_grad(set_to_none = True)
     loss.backward()
     optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist())) #[0] becasue we are taking the first element from generate's batch dim (albeit the batch dimension is only 1 rn)

Step 0: train loss 4.2041, val loss 4.2008
Step 500: train loss 2.7072, val loss 2.7276
Step 1000: train loss 2.5128, val loss 2.5303
Step 1500: train loss 2.4710, val loss 2.4869
Step 2000: train loss 2.4398, val loss 2.4633
Step 2500: train loss 2.4264, val loss 2.4498
Step 3000: train loss 2.4162, val loss 2.4442
Step 3500: train loss 2.3978, val loss 2.4310
Step 4000: train loss 2.4050, val loss 2.4059
Step 4500: train loss 2.3975, val loss 2.4169

When bef bridcowilen is by bte

Hiset bobe ule.
S:
O-d my dalilanss ar btharuu qure vet?
F dilas ate awice my.

ODER:
An orou wabuts, tof isth bot mil; dill, bes iree sen cie lat Het drov ts, and Wind.
GRWilerans!
el lind peal.
-hule onchiry ptugr aiss hew ye wllingu noroupetelaves homy ll, de telake onWindo wh theiiby, wist, fourive wen, ired thoouso er; th
Tow admy,
Tuptef so;
ARID Whe:
EN:
I inle ont ffaf Pre?

Whiom.

He-osup!
s,
Sho ak
Sadsal aces ghe thinin cour ay aney Iry ts I fr yf voul


In [18]:
# for steps in range(10000):
#     xb, yb = get_batch('train')
#     logits, loss = m(xb,yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step() #upgrade gradient
# print(loss.item())